In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import wandb

import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data

import networkx as nx
from networkx.convert_matrix import from_numpy_array
import hypernetx as hnx

import functions as f
from functions import dict_to_array, normalize_array
import models as m

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Loading the time series
time_series_list = f.load_time_series()
time_series_df_list = f.list_of_df_of_time_series(time_series_list)

In [3]:
# Loading the pearson correlation matrices
corr_matrix_list = f.corr_matrix_paths()['pearson']
print(corr_matrix_list)
# Example patient
path = f'ADNI_full/corr_matrices/corr_matrix_pearson/{corr_matrix_list[0]}'
corr_matrix_patient = np.loadtxt(path, delimiter=',')
print(corr_matrix_patient)

['patient_136_S_4517.csv', 'patient_130_S_4468.csv', 'patient_002_S_4654.csv', 'patient_002_S_4251.csv', 'patient_013_S_5171.csv', 'patient_130_S_2391.csv', 'patient_031_S_4947.csv', 'patient_100_S_5102.csv', 'patient_012_S_5121.csv', 'patient_053_S_5287.csv', 'patient_002_S_1261.csv', 'patient_018_S_4313.csv', 'patient_130_S_4641.csv', 'patient_012_S_4188.csv', 'patient_002_S_4521.csv', 'patient_130_S_4250.csv', 'patient_031_S_4005.csv', 'patient_130_S_5006.csv', 'patient_006_S_4363.csv', 'patient_012_S_4012.csv', 'patient_031_S_4149.csv', 'patient_031_S_4029.csv', 'patient_013_S_4917.csv', 'patient_129_S_4287.csv', 'patient_130_S_4730.csv', 'patient_129_S_0778.csv', 'patient_031_S_4203.csv', 'patient_013_S_4268.csv', 'patient_053_S_0919.csv', 'patient_002_S_4447.csv', 'patient_012_S_4987.csv', 'patient_018_S_4868.csv', 'patient_100_S_4556.csv', 'patient_100_S_5106.csv', 'patient_130_S_5175.csv', 'patient_130_S_4294.csv', 'patient_053_S_5296.csv', 'patient_031_S_4474.csv', 'patient_12

In [4]:
# Loading the diagnostic labels
diagnostic_label_all = np.loadtxt('ADNI_full/diagnostic_label.csv', dtype=str, delimiter=',')

In [5]:
time_series_folder = 'ADNI_full/time_series'

In [6]:
def filter_group(group):
    df = pd.read_csv('ADNI_FULL/patient_info.csv')
    labels = df['Research Group']
    label_idx_list = [i for i in range(len(labels)) if labels[i] == group]
    return label_idx_list

In [7]:
#binary classification
cn = filter_group('CN')
ad = filter_group('AD')
bin_idx = sorted(cn + ad)
corr_matrices = [corr_matrix_list[i] for i in bin_idx]
diagnostic_label = [diagnostic_label_all[i] for i in bin_idx]
for i in range(len(diagnostic_label)):
    if diagnostic_label[i] == 'CN':
        diagnostic_label[i] = 0
    elif diagnostic_label[i] == 'AD':
        diagnostic_label[i] = 1
    else:
        print('Error: incorrect label')

In [8]:
time_series_folder = 'ADNI_full/time_series'
time_series_folder_binary = []
for i, file_name in enumerate(os.listdir(time_series_folder)):
    if i in bin_idx:
        time_series_folder_binary.append(f'{time_series_folder}/{file_name}')
print(time_series_folder_binary)

['ADNI_full/time_series/ts_130_S_4883.csv', 'ADNI_full/time_series/ts_100_S_5091.csv', 'ADNI_full/time_series/ts_019_S_4549.csv', 'ADNI_full/time_series/ts_018_S_4257.csv', 'ADNI_full/time_series/ts_019_S_4367.csv', 'ADNI_full/time_series/ts_019_S_4548.csv', 'ADNI_full/time_series/ts_010_S_4442.csv', 'ADNI_full/time_series/ts_130_S_5142.csv', 'ADNI_full/time_series/ts_018_S_2180.csv', 'ADNI_full/time_series/ts_002_S_4262.csv', 'ADNI_full/time_series/ts_006_S_4150.csv', 'ADNI_full/time_series/ts_002_S_1280.csv', 'ADNI_full/time_series/ts_006_S_4346.csv', 'ADNI_full/time_series/ts_002_S_2010.csv', 'ADNI_full/time_series/ts_031_S_4024.csv', 'ADNI_full/time_series/ts_002_S_5178.csv', 'ADNI_full/time_series/ts_100_S_5280.csv', 'ADNI_full/time_series/ts_100_S_5096.csv', 'ADNI_full/time_series/ts_130_S_4660.csv', 'ADNI_full/time_series/ts_002_S_1268.csv', 'ADNI_full/time_series/ts_019_S_5242.csv', 'ADNI_full/time_series/ts_002_S_4270.csv', 'ADNI_full/time_series/ts_130_S_2373.csv', 'ADNI_full

In [9]:
assert len(diagnostic_label) == len(corr_matrices)

In [10]:
# architecture = 'CK_2Class'
architecture = 'FC_2Class'
# architecture = 'MC_2Class'
# architecture = 'KNN_2Class'

In [11]:
os.environ['WANDB_NOTEBOOK_NAME']=f'{architecture}.ipynb'

In [12]:
# Defining a class to preprocess raw data into a format suitable for training Graph Neural Networks (GNNs).
## With the possibility of assigning weight to edges, adding the age feature, sex feature, and matrixe profiling.

class Raw_to_Hypergraph_2Class(InMemoryDataset):
    def __init__(self, root, hg_data_path, method, weight, threshold, age=False, sex=False, transform=None, pre_transform=None):
        self.method = method
        self.weight = weight
        self.threshold = threshold
        self.age = age
        self.sex = sex
        self.hg_data_path = hg_data_path
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return ['data.pt']

    # This function is used to process the raw data into a format suitable for GNNs, by constructing graphs out of the connectivity matrices.
    def process(self):
        # Loading the prebuilt hypergraphs and the correlation matrices
        hg_dict_list = f.load_hg_dict(self.hg_data_path)

        graphs=[]
        for patient_idx, patient_matrix in enumerate(corr_matrices):
            # Create a NetworkX graph from the hypergraph matrix
            patient_hg = hg_dict_list[patient_idx]
            hypergraph = hnx.Hypergraph(patient_hg)

            # Adding the matrix profiling features to the feature array
            path = f'ADNI_full/matrix_profiles/matrix_profile_pearson/{patient_matrix}'
            if patient_matrix.endswith('.DS_Store'):
                continue  # Skip hidden system files like .DS_Store
            with open(path, "rb") as fl:
                patient_dict = pkl.load(fl)
            # combine dimensions
            features = np.array(patient_dict['mp']).reshape(len(patient_dict['mp']),-1)
            features = features.astype(np.float32)

            path_corr = f'ADNI_full/corr_matrices/corr_matrix_pearson/{patient_matrix}'
            corr_array = np.loadtxt(path_corr, delimiter=',')  
            
            # Concatenate the matrix profiling features to the feature array
            x_array = np.concatenate((corr_array, features), axis=-1)
            x_array = x_array.astype(np.float32)
            x = torch.tensor(x_array, dtype=torch.float)

            # Create a Pytorch Geometric Data object
            edge_index0 = []
            edge_index1 = []
            i = 0
            for hyperedge, nodes in hypergraph.incidence_dict.items():
                edge_index0 = np.concatenate((edge_index0, nodes), axis=0)
                for j in range(len(nodes)):
                    edge_index1.append(i)
                i += 1
            edge_index = np.stack([[int(x) for x in edge_index0], edge_index1], axis=0)
            y = torch.tensor(float(diagnostic_label[patient_idx]))
            hg_data = Data(x=x, edge_index=torch.tensor(edge_index, dtype=torch.long), y=y)
            graphs.append(hg_data)

        data, slices = self.collate(graphs)
        torch.save((data, slices), self.processed_paths[0])

In [13]:
assert len(corr_matrices) == len(time_series_folder_binary)

In [14]:
corr_matrices[0]

'patient_136_S_4517.csv'

In [23]:
# Building the Hypergraphs (choose one method cell above)
# method_list = ['maximal_clique', 'knn']
method_list = ['fourier_cluster']
m.save_all_hypergraphs(method_list, time_series_folder_binary, corr_matrices, diagnostic_label)

Processing patient 130_S_4883


/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

Patient 130_S_4883 processed and saved for the coskewness method with threshold 0.5
Processing patient 100_S_5091


/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

Patient 100_S_5091 processed and saved for the coskewness method with threshold 0.5
Processing patient 019_S_4549


...

/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

Patient 031_S_4947 processed and saved for the coskewness method with threshold 0.5
Processing patient 130_S_4468


/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

Patient 130_S_4468 processed and saved for the coskewness method with threshold 0.5
Processing patient 002_S_4654
Patient 002_S_4654 processed and saved for the coskewness method with threshold 0.5


/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

In [25]:
# Doing some parameter gridsearch to find the best hyperparameters
from sklearn.model_selection import ParameterGrid

# Building the graphs
threshold = 0.5
k_neighbors = 3
age = False
sex = False
method = 'fourier_cluster'
if method == 'knn':
    threshold = f'{k_neighbors}neighbors'
weight = False

hg_data_path = f'Hypergraphs/{method}/thresh_{threshold}'
root = f'Raw_to_hypergraph_2Class/ADNI_T_{threshold}_M_{method}_W{weight}_A{age}_S{sex}_MPTrue'
dataset = Raw_to_Hypergraph_2Class(root=root, hg_data_path=hg_data_path, method=method, weight=weight, threshold=threshold, age=age, sex=sex)

# Creating the train, validation and test sets
train_loader, valid_loader, test_loader, nbr_classes = f.create_train_test_valid(dataset)

# param_grid = {
#     'learning_rate': [0.001, 0.0001],
#     'hidden_channels': [128, 64],
#     'num_layers': [3, 2, 1],
#     'dropout_rate': [0.2, 0.1, 0.0],
#     'weight_decay': [0.001, 0.0001]
# }
param_grid = {
    'learning_rate': [0.0001, 0.001],
    'hidden_channels': [64, 128],
    'num_layers': [1, 2, 3],
    'dropout_rate': [0.0, 0.1, 0.2],
    'weight_decay': [0.0001, 0.001]
}

# Create combinations of hyperparameters
param_combinations = ParameterGrid(param_grid)
architecture = architecture
n_epochs = 800
in_channels = dataset.num_features
# Train using each combination
for params in param_combinations:
    filename = f'2Class_Models/HGConv_Models_MP/threshold_{threshold}/method_{method}/lr{params["learning_rate"]}_hc{params["hidden_channels"]}_nl{params["num_layers"]}_d{params["dropout_rate"]}_epochs{n_epochs}_wdecay{params["weight_decay"]}_w{weight}.png'
    if os.path.exists(filename):
        pass
    else:
        parameters = [params['learning_rate'], params['hidden_channels'], params['num_layers'], params['dropout_rate']]
        model = m.HGConv(in_channels=in_channels, hidden_channels=parameters[1], out_channels=nbr_classes, num_layers=parameters[2], dropout=parameters[3], nbr_classes=nbr_classes)
        criterion = torch.nn.CrossEntropyLoss() 
        if 'weight_decay' not in params.keys():
            w_decay = 0
        else:
            w_decay = params['weight_decay']
        optimizer = torch.optim.Adam(model.parameters(), lr=parameters[0], weight_decay=w_decay)
        train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy, test_accuracy = m.train_HGConv(model, optimizer, criterion, w_decay, threshold, train_loader, valid_loader, parameters, architecture, test_loader, testing=True, n_epochs=800)


Processing...
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = 

Number of training graphs: 54
Number of validation graphs: 8
Number of test graphs: 16
Number of classes: 2


Done!
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Max Valid Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▃▃▁▂▃▃▃▃▂▃▃▄▄▃▅▃▄▆▆▄▃▇▆▆▅▆▆▇▇▇▆▇▇▇█▇████
Train Loss,▆█▆▆▆▆▆▅▆▆▅▅▅▆▅▅▄▄▄▄▄▄▃▄▄▃▃▃▂▃▃▂▂▂▂▂▂▁▁▂
Validation Accuracy,██▆▆█▃▁█▃▃▃▃▃██▃▆▃████████▃▆█▆██▆██▃▃█▆█
Validation Loss,█▇▃▃▅▂▂▅▂▂▂▂▂█▅▂▂▂▃▃▃▃▅▃▅▅▂▁▅▂▃▂▁▅▃▁▁▃▂▆
Max Valid Accuracy,0.5
Test Accuracy,0.5625
Train Accuracy,1.0
Train Loss,0.22436
Validation Accuracy,0.375


/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


tensor([[  17,   33,   17,  ..., 1847, 1854, 1854],
        [   0,    0,    1,  ..., 2169, 2169, 2170]])
tensor([[   5,   12,   33,  ..., 1816, 1817, 1853],
        [   0,    0,    0,  ..., 1762, 1762, 1762]])
tensor([[   6,   12,   27,  ..., 1852, 1824, 1834],
        [   0,    0,    0,  ..., 2262, 2263, 2263]])
tensor([[ 17,  88,  20,  ..., 622, 675, 688],
        [  0,   0,   1,  ..., 589, 589, 589]])
tensor([[  0,   5,   9,  ..., 905, 895, 913],
        [  0,   0,   0,  ..., 871, 872, 872]])
tensor([[  22,   51,  115,  ..., 1854, 1852, 1855],
        [   0,    0,    0,  ..., 2185, 2186, 2186]])
Epoch 1/800
Train Loss: 0.8085, Validation Loss: 0.9530
Train Accuracy: 0.4844, Validation Accuracy: 0.5000
Max Validation Accuracy: 0.5000
tensor([[   2,   22,    2,  ..., 1853, 1842, 1853],
        [   0,    0,    1,  ..., 1881, 1882, 1882]])
tensor([[   8,    9,    8,  ..., 1838, 1840, 1842],
        [   0,    0,    1,  ..., 1746, 1746, 1746]])
tensor([[   6,    6,   49,  ..., 1843, 1844,

KeyboardInterrupt: 

wandb: Network error (ConnectionError), entering retry loop.
